## TF보기
tf.keras : tf을 쉽게.
tf.contrib : 케라스를 더 쉽게.

In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense,TimeDistributed,SimpleRNN,LSTM,Activation
from keras.utils import np_utils # Numpy 안에 utils 호출
import os

Using TensorFlow backend.


In [2]:
from keras.utils.vis_utils import plot_model # plot 확인을 위한 모듈

In [3]:
#smaple text
sample = 'hihello'

In [4]:
# word dict 만들기
char_set = list(set(sample)) # id -> char['i','l','e','h','o']
char_dic = {w : i for i , w in enumerate(char_set)}
char_dic

{'h': 0, 'e': 1, 'o': 2, 'l': 3, 'i': 4}

In [5]:
# 학습용 데이터 만들기 
x_str = sample[:-1] # input, 맨 마지막 값은 없이.
y_str = sample[1:] # output, 첫번째 입력값은 없이.

In [6]:
data_dim = len(char_set) # 데이터 구조의 길이
timesteps = len(y_str) # y의 개수를 읽어서 미리 예측, 시간을 입력해놓음
num_classes = len(char_set)

In [7]:
print(x_str,y_str)
x = [char_dic[c] for c in x_str]
y = [char_dic[c] for c in y_str]

hihell ihello


In [8]:
# one - hot encoding
x = np_utils.to_categorical(x,num_classes = num_classes)
print(x)
#reshape X to be [samples,time steps, features]
x = np.reshape(x,(-1,len(x),data_dim)) # -1 : 최대한 읽어들이고 / len,data_dim 만큼 한번에 입력.
print(x.shape)  # (data의 개수,timestep,벡터의 크기)

[[1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]]
(1, 6, 5)


In [9]:
y = np_utils.to_categorical(y,num_classes = num_classes)
y = np.reshape(y,(-1,len(y),data_dim))
print(y.shape)
print(y) # 3차원 배열으로 변신

(1, 6, 5)
[[[0. 0. 0. 0. 1.]
  [1. 0. 0. 0. 0.]
  [0. 1. 0. 0. 0.]
  [0. 0. 0. 1. 0.]
  [0. 0. 0. 1. 0.]
  [0. 0. 1. 0. 0.]]]


In [10]:
# vanila SimpleRNN
model = Sequential()
model.add(LSTM(num_classes,input_shape=(  # num_classes 는 hidden layer의 노드 개수로 지정(one-hot의 계수값)
timesteps,data_dim),return_sequences=True))  # 6,5 의 구조로 입력 
model.add(Dense(num_classes))
model.add(Activation('softmax'))
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 6, 5)              220       
_________________________________________________________________
dense_1 (Dense)              (None, 6, 5)              30        
_________________________________________________________________
activation_1 (Activation)    (None, 6, 5)              0         
Total params: 250
Trainable params: 250
Non-trainable params: 0
_________________________________________________________________


In [11]:
model.compile(loss='categorical_crossentropy',
             optimizer = 'adam', metrics=['accuracy'])
model.fit(x,y,epochs=1,batch_size=1) 
# 2개의 차원이여야하는데 y가 3차원으로 들어와서 에러 발생


Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Epoch 1/1
1/1 [==============================] - 2s 2s/step - loss: 1.6352 - acc: 0.1667


In [12]:
predictions = model.predict(x,verbose=0)

In [13]:
for i,prediction in enumerate(predictions):
    print(prediction)
    x_index = np.argmax(x[i],axis=1)
    x_str = [char_set[j] for j in x_index]
    print(x_index,''.join(x_str))
    
    index = np.argmax(prediction,axis=1)
    result = [char_set[j] for j in index]
    print(index,''.join(result))
print(x_str)

[[0.20592304 0.17911749 0.22949092 0.18792568 0.1975429 ]
 [0.22683398 0.16472787 0.22995947 0.17855151 0.19992717]
 [0.21731678 0.1600705  0.2520188  0.17835566 0.19223833]
 [0.2073616  0.17752913 0.23435663 0.18319471 0.19755788]
 [0.20130841 0.17130901 0.24638201 0.16959374 0.21140677]
 [0.19938524 0.17050123 0.24843967 0.16176145 0.2199124 ]]
[0 4 0 1 3 3] hihell
[2 2 2 2 2 2] oooooo
['h', 'i', 'h', 'e', 'l', 'l']
